<a href="https://colab.research.google.com/github/lazaroq11/bill_model/blob/master/bill_qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [5]:
!pip install transformers
!pip install -U scikit-learn
!pip install pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 106.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [8]:
!pip install transformers[torch]
!pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 7.0 MB/s eta 0:00:00


In [3]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.0 MB/s eta 0:00:00


In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader

# Seu DataFrame com textos e rótulos
import pandas as pd
import random
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
from textblob import Word

# Seu DataFrame com textos e rótulos
df = pd.DataFrame({
    "label": ["BILL", "BahiaRT", "python","AI", "Machine Learning", "NLP", "ACSO", "UNEB", "BahiaRt Multidisciplinar","Yolo", "ROS"],
    "text": ["BILL é um robô de serviço projetado pelo teamBahiaRt. Ele é um robô humanoide que pode ser usado para uma variedade de tarefas, incluindo atendimento ao cliente, limpeza e segurança. Bill é equipado com uma variedade de sensores e câmeras, que ele usa para navegar pelo ambiente e interagir com humanos.Bill é um produto da BahiaRT, uma equipe de robótica da UNEB. A BahiaRT é uma equipe de estudantes e pesquisadores que trabalham no desenvolvimento de robôs para aplicações sociais. Bill é um dos projetos mais recentes da BahiaRT.Bill é um robô inovador que tem o potencial de melhorar a vida das pessoas. Ele pode ser usado para automatizar tarefas, fornecer assistência aos idosos e deficientes e até mesmo ajudar a salvar vidas.",
              "BahiaRT é uma equipe de robótica da UNEB. A equipe foi fundada em 2015 por um grupo de estudantes e pesquisadores que compartilham o interesse em robótica.",
              "Python é uma linguagem de programação geral de alto nível. É uma linguagem interpretada, o que significa que não precisa ser compilada antes de ser executada. Python é uma linguagem relativamente fácil de aprender e usar.",
              "A inteligência artificial (IA) é uma área da ciência da computação que se concentra no desenvolvimento de sistemas capazes de executar tarefas que normalmente requerem inteligência humana. Ela engloba campos como aprendizado de máquina, visão computacional e processamento de linguagem natural.",
              "A aprendizagem de máquina (Machine Learning) é um subcampo da inteligência artificial que se concentra no desenvolvimento de algoritmos e modelos que permitem que os sistemas aprendam e melhorem com base em dados. É amplamente utilizado em previsões e análise de dados.",
              "O Processamento de Linguagem Natural (NLP) é uma subárea da IA que se concentra na interação entre computadores e linguagem humana. É usado em chatbots, tradução automática e análise de sentimentos, entre outras aplicações.",
              "O ACSO (Núcleo de Arquitetura de Computadores e Sistemas Operacionais), é um grupo de Pesquisa da UNEB, Universidade do Estado da Bahia",
              "A Uneb é uma das maiores universidades da Bahia, multicampi e para todos.",
              "A BahiaRT é uma equipe multidisciplinar que inclui estudantes de engenharia, ciência da computação e design.A BahiaRT tem trabalhado em uma variedade de projetos de robótica, incluindo Bill, um robô de serviço humanoide. A equipe também está trabalhando em um robô de resgate que pode ser usado em situações de emergência.A BahiaRT é uma equipe ativa que está constantemente desenvolvendo novos projetos de robótica. A equipe está comprometida em usar a robótica para melhorar a vida das pessoas.",
              "O Yolo é um algoritmo de AI, utilizado para detecção de pessoas e objetos, BILL utiliza esse algoritmo para executar tarefas relacionadas a isso",
              "Robot Operating System é uma coleção de frameworks de software para desenvolvimento de robôs, que fornece a funcionalidade de um sistema operacional em um cluster de computadores heterogêneo "]
})

# Função para substituir palavras por sinônimos
def replace_with_synonyms(text):
    words = text.split()
    for i in range(len(words)):
        if random.random() < 0.2:  # Probabilidade de 20% de substituição
            # Obter sinônimos da palavra usando Wordnet
            synonyms = wordnet.synsets(words[i])
            if synonyms:
                new_word = synonyms[0].lemmas()[0].name()
                words[i] = new_word
    return ' '.join(words)

# Aplicar a função de substituição de palavras por sinônimos ao DataFrame
df['text'] = df['text'].apply(replace_with_synonyms)

# Dividir o DataFrame em treinamento e teste
train_df, test_df = train_test_split(df, test_size=0.2)

# Carregar o tokenizador e modelo pré-treinado GPT-2
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large')
model = GPT2LMHeadModel.from_pretrained('gpt2-large')

# Defina um token de padding
tokenizer.pad_token = tokenizer.eos_token

# Defina o tamanho do lote e crie um DataLoader personalizado para treinamento
class TextDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.texts = df['text'].values
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        inputs = self.tokenizer(self.texts[idx], return_tensors='pt', padding='max_length', max_length=128, truncation=True)
        return inputs

train_dataset = TextDataset(train_df, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# Configurar otimizador e agendador de taxa de aprendizado
optimizer = AdamW(model.parameters(), lr=2e-4)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * 10)

# Adicione dropout como técnica de regularização
model.config.dropout = 0.1

# Treinamento
model.train()
for epoch in range(10):
    total_loss = 0.0
    for batch in train_dataloader:
        optimizer.zero_grad()
        inputs = batch["input_ids"]
        loss = model(inputs, labels=inputs).loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        total_loss += loss.item()
    print(f'Epoch {epoch + 1}, Loss: {total_loss / len(train_dataloader)}')

    # Gere texto de exemplo após algumas épocas
    if (epoch + 1) % 10 == 0:
        model.eval()
        prompt = "BILL"
        prompt_ids = tokenizer(prompt, return_tensors='pt').input_ids
        generated_text = model.generate(prompt_ids, max_length=100, num_return_sequences=1, no_repeat_ngram_size=2)
        print(f'Texto gerado na época {epoch + 1}:')
        print(tokenizer.decode(generated_text[0], skip_special_tokens=True))
        model.train()

# Salvar o modelo fine-tunado
model.save_pretrained("fine_tuned_gpt2")
tokenizer.save_pretrained("fine_tuned_gpt2")


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Loss: 5.245918154716492
Epoch 2, Loss: 1.9514864087104797
Epoch 3, Loss: 1.2550402283668518
Epoch 4, Loss: 0.8739748299121857
Epoch 5, Loss: 0.5643481314182281
Epoch 6, Loss: 0.3487187922000885
Epoch 7, Loss: 0.20729544758796692
Epoch 8, Loss: 0.12430823594331741
Epoch 9, Loss: 0.061371784657239914


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Epoch 10, Loss: 0.04847209341824055
Texto gerado na época 10:
BILL ACSO (Núcleo de Arquitetura de Computadores e Sistemas Operacionais), é um grupo de Pesquisa district_attorney UNEB, Universidade do Estado da Bahia


('fine_tuned_gpt2/tokenizer_config.json',
 'fine_tuned_gpt2/special_tokens_map.json',
 'fine_tuned_gpt2/vocab.json',
 'fine_tuned_gpt2/merges.txt',
 'fine_tuned_gpt2/added_tokens.json')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [34]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Carregar o modelo fine-tunado
fine_tuned_model = GPT2LMHeadModel.from_pretrained("fine_tuned_gpt2")
fine_tuned_tokenizer = GPT2Tokenizer.from_pretrained("fine_tuned_gpt2")

# Exemplo de inferência com temperatura definida
prompt = "O que é o ACSO"
input_ids = fine_tuned_tokenizer.encode(prompt, return_tensors='pt')
temperature = 0.6  # Ajuste o valor de temperatura conforme desejado
with torch.no_grad():
    output = fine_tuned_model.generate(input_ids, max_length=100, num_return_sequences=1, no_repeat_ngram_size=2, temperature=temperature)
generated_text = fine_tuned_tokenizer.decode(output[0], skip_special_tokens=True)

print("Texto gerado:")
print(generated_text)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Texto gerado:
O que é o ACSO (Núcleo de Arquitetura de Computadores e Sistemas Operacionais), é um grupo de Pesquisa district_attorney UNEB, Universidade do Estado da Bahia


In [25]:
!pip install nltk

In [28]:
import pandas as pd
import random
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
from textblob import Word

# Seu DataFrame com textos e rótulos
df = pd.DataFrame({
    "label": ["BILL", "BahiaRT", "python","AI", "Machine Learning", "NLP", "ACSO", "UNEB", "BahiaRt Multidisciplinar","Yolo", "ROS"],
    "text": ["BILL é um robô de serviço projetado pelo teamBahiaRt. Ele é um robô humanoide que pode ser usado para uma variedade de tarefas, incluindo atendimento ao cliente, limpeza e segurança. Bill é equipado com uma variedade de sensores e câmeras, que ele usa para navegar pelo ambiente e interagir com humanos.Bill é um produto da BahiaRT, uma equipe de robótica da UNEB. A BahiaRT é uma equipe de estudantes e pesquisadores que trabalham no desenvolvimento de robôs para aplicações sociais. Bill é um dos projetos mais recentes da BahiaRT.Bill é um robô inovador que tem o potencial de melhorar a vida das pessoas. Ele pode ser usado para automatizar tarefas, fornecer assistência aos idosos e deficientes e até mesmo ajudar a salvar vidas.",
              "BahiaRT é uma equipe de robótica da UNEB. A equipe foi fundada em 2015 por um grupo de estudantes e pesquisadores que compartilham o interesse em robótica.",
              "Python é uma linguagem de programação geral de alto nível. É uma linguagem interpretada, o que significa que não precisa ser compilada antes de ser executada. Python é uma linguagem relativamente fácil de aprender e usar.",
              "A inteligência artificial (IA) é uma área da ciência da computação que se concentra no desenvolvimento de sistemas capazes de executar tarefas que normalmente requerem inteligência humana. Ela engloba campos como aprendizado de máquina, visão computacional e processamento de linguagem natural.",
              "A aprendizagem de máquina (Machine Learning) é um subcampo da inteligência artificial que se concentra no desenvolvimento de algoritmos e modelos que permitem que os sistemas aprendam e melhorem com base em dados. É amplamente utilizado em previsões e análise de dados.",
              "O Processamento de Linguagem Natural (NLP) é uma subárea da IA que se concentra na interação entre computadores e linguagem humana. É usado em chatbots, tradução automática e análise de sentimentos, entre outras aplicações.",
              "O ACSO (Núcleo de Arquitetura de Computadores e Sistemas Operacionais), é um grupo de Pesquisa da UNEB, Universidade do Estado da Bahia",
              "A Uneb é uma das maiores universidades da Bahia, multicampi e para todos.",
              "A BahiaRT é uma equipe multidisciplinar que inclui estudantes de engenharia, ciência da computação e design.A BahiaRT tem trabalhado em uma variedade de projetos de robótica, incluindo Bill, um robô de serviço humanoide. A equipe também está trabalhando em um robô de resgate que pode ser usado em situações de emergência.A BahiaRT é uma equipe ativa que está constantemente desenvolvendo novos projetos de robótica. A equipe está comprometida em usar a robótica para melhorar a vida das pessoas.",
              "O Yolo é um algoritmo de AI, utilizado para detecção de pessoas e objetos, BILL utiliza esse algoritmo para executar tarefas relacionadas a isso",
              "Robot Operating System é uma coleção de frameworks de software para desenvolvimento de robôs, que fornece a funcionalidade de um sistema operacional em um cluster de computadores heterogêneo "]
})

# Função para substituir palavras por sinônimos
def replace_with_synonyms(text):
    words = text.split()
    for i in range(len(words)):
        if random.random() < 0.2:  # Probabilidade de 20% de substituição
            # Obter sinônimos da palavra usando Wordnet
            synonyms = wordnet.synsets(words[i])
            if synonyms:
                new_word = synonyms[0].lemmas()[0].name()
                words[i] = new_word
    return ' '.join(words)

# Aplicar a função de substituição de palavras por sinônimos ao DataFrame
df['text'] = df['text'].apply(replace_with_synonyms)

print(df['text'])

# Agora, o DataFrame contém amostras com palavras substituídas por sinônimos


0     bill é um robô Delaware serviço projetado pelo...
1     BahiaRT é uma equipe de robótica da UNEB. A eq...
2     Python é uma linguagem de programação geral de...
3     A inteligência artificial (IA) é uma área da c...
4     A aprendizagem de máquina (Machine Learning) é...
5     O Processamento de Linguagem Natural (NLP) é u...
6     O ACSO (Núcleo de Arquitetura de Computadores ...
7     A Uneb é uma das maiores universidades distric...
8     A BahiaRT é uma equipe multidisciplinar que in...
9     O Yolo é um algoritmo Delaware AI, utilizado p...
10    Robot Operating System é uma coleção de framew...
Name: text, dtype: object


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
